## Afternoon practical day 2

Welcome to the final practical of today. Here you will implement multi-class logistic regression before moving on to forward propagation for your very own neural network. For the latter, we'll be working with the well-known MNIST dataset that contains images of handwritten digits that need to be classified. First, run the two cells below, and work on the 2D dataset in which you can clearly show the decision boundary learned.



In [ ]:
#run this cell to set things up
import ipywidgets as widgets, numpy as np, pandas as pd
from numpy.random import default_rng
%matplotlib inline
import matplotlib.pyplot as plt
import math
import seaborn as sns
from IPython.display import display, Markdown
import scipy
from scipy.optimize import fmin_bfgs, fmin_cg, fmin
import sklearn

In [ ]:
#important functions
def mySigmoid(data):
    data= np.array(data)
    output = 1/(1+ np.exp(-data))
    return output

def linAlgRegHypothesis(data, thetas):
    data = np.array(data)
    oneFeatToAdd = np.ones(len(data))
    newFeatArray = np.c_[oneFeatToAdd, data]
    #make sure thetas are always of the form np.array([[theta1], [theta2]]), i.e. column vector
    if thetas.ndim < 2:
        thetas = thetas[:, np.newaxis]
    predictions = newFeatArray @ thetas
    #print(predictions)
    return predictions

def linAlgLogRegHypothesis(data, thetas):
    output = mySigmoid(linAlgRegHypothesis(data, thetas))
    return output

def costFuncLogReg(thetas, x, y, lambda_ = 0, smallValueToPreventInfinity = 1e-12):
    m = len(x)
    predictions      = linAlgLogRegHypothesis(x, thetas)
    #print(predictions+smallValueToPreventInfinity)
    #print(-y * np.log(predictions+smallValueToPreventInfinity))
    #print(-y * np.log(predictions))
    costsPerSample   = -y * np.log(predictions+smallValueToPreventInfinity) - (1-y) * np.log(1 - (predictions+smallValueToPreventInfinity))
    #set the bias to 0 so you don't count it in regularisation.
    #It is convention not to regularise the bias/intercept
    thetas[0]        = 0
    regCost          = lambda_/2 * 1/m * thetas.T @ thetas
    totalCosts       = 1/m * np.nansum(costsPerSample) + regCost
    totalCosts       = np.ravel(totalCosts)
    #print(totalCosts)
    return totalCosts

def computeGradients(thetas, x, y, lambda_ = 0, hypothesis = 'linAlgLogRegHypothesis'):
    m = len(x)
    if thetas.ndim < 2:
        thetas = thetas[:, np.newaxis]
    preds  = globals()[hypothesis](x, thetas)
    if preds.shape != (m, 1):
        preds  = preds[:, np.newaxis]
    if y.ndim < 2:
        y = y[:, np.newaxis]
    errors = preds - y
    gradientSummation  = errors.T @ np.c_[np.ones(len(errors)), x]
    unregularisedGradients  = 1/m * gradientSummation
    regularisedGradients    = np.ravel(unregularisedGradients)
    #print(regularisedGradients)
    regularisedGradients[1:]    = regularisedGradients[1:] + lambda_/m * np.ravel(thetas)[1:]
    #print(regularisedGradients)
    return regularisedGradients

def gradientDescentStep(thetas, gradients, alpha):
    if thetas.ndim < 2:
        thetas = thetas[:, np.newaxis]
    if gradients.ndim < 2:
        gradients = gradients[:, np.newaxis]
    newThetas = thetas - alpha * gradients
    return newThetas

def makeCrossValData(dataFrame, k=10):
    '''function to make splits into training and validation sets.
    Outputs two lists of length k, where each element is the indices of samples to train on for that fold, 
    and the indices of samples to test on for that fold, respectively.'''
    #shuffle data
    dataFrame = dataFrame.sample(frac=1)
    m = len(dataFrame)
    #see how many equal-sized sets you can make
    dataPerSplit = int(np.floor(m/k))
    dataPartitions = []
    counter = 0

    for i in range(0,k):
        #make a list of all the samples for each fold
        dataPartitions.append(list(range(counter,counter+dataPerSplit)))
        counter += dataPerSplit

    samplesEquallySplit = k * dataPerSplit
    if not samplesEquallySplit == m:
        #after making equal splits there will be samples left, i.e. you cannot always make k exactly evenly sized subsets.
        #randomly assign left over samples to folds after
        toDivide = m-samplesEquallySplit
        for extraSampleIndex in range(counter, counter+toDivide):
            #only assign to lists of samples that have the current minimum amount of samples
            currentSubsetSizes = np.array([len(subset) for subset in dataPartitions])
            assignTo = np.random.choice(np.where(currentSubsetSizes == np.min(currentSubsetSizes))[0])
            dataPartitions[assignTo].append(extraSampleIndex)
    
    #Now make the final cross-validation set: make k sets, each set has (k-1)/k folds to train on, and 1 fold to test on.
    testSet = []
    trainSet = []
    for validationSetIndex in range(0,k):
        #put 1 fold in the test set
        testSet.append(dataPartitions[validationSetIndex])
        #put all other folds in the train set
        trainSet.append(dataPartitions.copy())
        trainSet[validationSetIndex].pop(validationSetIndex)
        #this line makes sure all training set indices are in one big list, rather than k-1 small lists. 
        trainSet[validationSetIndex] = [item for sublist in trainSet[validationSetIndex] for item in sublist]
    
    return dataFrame, trainSet, testSet



def createNormalisedFeatures(dataFrame, mode = "range"):
    featureMeans = dataFrame.mean()
    if mode == "range":
        featureRanges = dataFrame.max() - dataFrame.min()
        normalisedFeatures = (dataFrame - featureMeans)/featureRanges
        return [normalisedFeatures, featureMeans, featureRanges]
    elif mode == "SD":
        featureSDs = dataFrame.std()
        normalisedFeatures = (dataFrame - featureMeans)/featureSDs
        return [normalisedFeatures, featureMeans, featureSDs]
    return None

def getDecisionBoundaryYValues(feat1, thetas):
    """Returns the y values for a decision boundary based on thetas for a 2D logistic regression problem"""
    thetas = np.ravel(thetas)
    b, w1, w2 = thetas
    intercept = -b/w2
    gradient  = -w1/w2
    yValues   = gradient * feat1 + intercept
    return yValues

## Multiclass (one-vs-all) classification with logistic regression

For multiclass classification the labels are no longer just 1 or 0, but can be from 0 to _n_.

During training, you loop over the number of labels, training a classifier for each. So you train a classifier to discern class 1 from not-1, class 2 from not-2, etc. Below, I generate some data for you to work on and plot it. After that, you're asked to train 5 classifiers on this data. 

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

features, labels = make_blobs(n_samples = 500, n_features = 2, centers = 5, center_box = (-8, 8), 
                             random_state = 42)

#make train and test set
#make sure that the proportion of each class in the train and test set is equal
featsTrain, featsTest, labelsTrain, labelsTest = train_test_split(features, labels, test_size = 0.2, stratify = labels)
#normalise the data
featsTrain, muFeatsTrain, rangeFeatsTrain = createNormalisedFeatures(pd.DataFrame(featsTrain), "range")
featsTrain = np.array(featsTrain)
featsTest  = (np.array(featsTest) - np.array(muFeatsTrain))/np.array(rangeFeatsTrain)
#note how you normalise the training data, and then use the obtained means and ranges to transform the test data!

#Show the data
dataFrameFiveClasses = pd.concat([pd.DataFrame(featsTrain), pd.DataFrame(labelsTrain)], axis = 1)
dataFrameFiveClasses.columns   = ["APRX-Y expression", "MYC expression", "Class"]
display(dataFrameFiveClasses.head())

#Make a plot of the training data using Seaborn
fig, ax = plt.subplots(figsize = (7,7))
sns.scatterplot(x = "APRX-Y expression", y = "MYC expression", hue = "Class", data = dataFrameFiveClasses,
               palette= "Dark2", ax = ax)



## Doing multiclass logistic regression

Now it is up to you to perform multiclass logistic regression. For now let's not use regularisation. To do this, do the following:

* Make a list of thetas `finalThetasEachClass`.
* Loop over the labels (there should be 5 iterations)
* In each loop:
    * Make a new class array, that is 1 for the current class under consideration, and 0 for all others (use the training data).
    * Use this class array to train logistic regression. Rather than using gradient descent, use `fmin_bfgs(costFunction, startThetas, gradientFunction, (features, class))`.
    * Append the final thetas to the list.
* When that's done, use `np.c_[yourListName].T` on the list to get a matrix of thetas, where each column contains the thetas for a certain class, with the first row containing $\theta_0$, the second containing $\theta_1$ etc.
* Which class has the highest cost?
* Finally, uncomment the plotting code to see what decision boundary has been learned!

Hints:
* Make sure that your class labels are a 2D array, that is, look like this: $\begin{bmatrix} 0 \\ 1 \\ 1 \\ ... \\ 1 \\ 0 \end{bmatrix}$
* In the answers, I've drawn the decision boundaries as well, you don't need to do that here (but you can look at them).

In [ ]:
startThetas = np.array([0]*3) #2 features and intercept

# your answer




# uncomment the below for your final output

# thetasAllClasses = np.c_[SOMETHING].T


# plotting code. Once you have thetasAllClasses,
# uncomment this code to generate plots of the decision boundaries learned

# xx, yy = np.mgrid[-1:1:.001, -1:1:.001]
# grid = np.c_[xx.ravel(), yy.ravel()]

# for classNumber in range(0, thetasAllClasses.shape[1]):
#     thisClassIndices    = np.where(labelsTrain == classNumber)
#     notThisClassIndices = np.where(labelsTrain != classNumber)
#     probs = linAlgLogRegHypothesis(grid, finalThetasEachClass[classNumber]).reshape(xx.shape) 
#     fig, ax = plt.subplots(figsize = (8, 8))
#     b = ax.contour(xx, yy, probs, levels=[.5], cmap="Greys", vmin=0, vmax=1)
#     ax.set_title("Class of interest: " + str(classNumber))
#     ax.set_xlabel("Feature 1")
#     ax.set_ylabel("Feature 2")
#     ax.scatter(featsTrain[thisClassIndices[0],0], featsTrain[thisClassIndices[0],1], label = "this class")
#     ax.scatter(featsTrain[notThisClassIndices[0],0], featsTrain[notThisClassIndices[0],1], label = "other classes")
#     b.collections[0].set_label("Learned decision boundary")
#     ax.legend()



## Making predictions with your learned decision boundaries

We didn't split our data into train and test subsets for nothing. We want to make some predictions! Luckily that's extremely easy. Look:

$$\begin{bmatrix}1 & 0.2 & 3 \\ 1 & -1.4 & 0.09 \end{bmatrix} \cdot \begin{bmatrix}\theta_0^1 & \theta_0^2 & \theta_0^3 \\ \theta_1^1 & \theta_1^2 & \theta_1^3 \\ \theta_2^1 & \theta_2^2 & \theta_2^3 \end{bmatrix}$$

Here, the superscript refers to the class, and the subscript to the parameter number. So $\theta_1^1$ is parameter 1 for the binary predicting whether something is class 1 or not class 1. You see that you'll get a 2 by 3 matrix as output (2 by 3 * 3 by 3 = 2 by 3), with on each row the output of the 3 separate classifiers in each column. To get your prediction, you simply select the maximum of each row, and that's the class you predict!

You don't even have to do that manually, as your `linAlgLogRegHypothesis` should do it for you. Up to you to:
* Predict the probability of membership of each class for the `featsTest`. Use `thetasAllClasses` from above.
* Select the class with the highest probability, and set this as your predicted class label. You can get the highest entries per row with `np.amax(yourData, axis = 1)[:,np.newaxis]`. You should use `np.where`.
* Compare this with `labelsTest` and see what percentage you predict correctly.

Hints
* When using `np.where`, make sure you get only an array of labels, not the row positions as well (you need to subset).

In [ ]:
# your answer here



## Multiclass classification on MNIST

It's nice to have a baseline of performance before we move on to a neural network. So we can also run a multiclass logistic regression on the MNIST dataset. This means that in this case you get a matrix of theta values, one set for each class. The MNIST data consists of 8 \* 8 grayscale images, i.e. 64 values between 0-16 that indicate the intensity of a pixel (0=black, 16 = white). To make this matrix of values into something logistic regression can work with, we simply paste the rows of pixel intensities one after the other into a 1 by 64 row vector. After the 10 classifiers have been trained, you predict with all of them on the test data, and then simply pick the highest probability among them as the class to predict for that digit.

Below we load in the data and show some digits. After that, you're asked to train the 10 classifiers you need and predict on the test set.

In [ ]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
mnistData = load_digits()
print(mnistData.DESCR)
print("\n________________________________________\n")

print("Amount of samples per digit:")
display(pd.DataFrame(np.unique(mnistData.target, return_counts = True)))

#plot first 16 images
fig, axes = plt.subplots(4, 4, figsize = (8,8))
counter = 0
for imageRow in range(0,4):
    for imageCol in range (0,4):
        axes[imageRow, imageCol].imshow(mnistData.images[counter], cmap='gray', vmin = 0, vmax = 16, interpolation='none')
        counter += 1

#split the data into train and test, keeping class percentage equal, using a function from scikit-learn
X, y                                          = load_digits(return_X_y= True)
X_train, X_test, y_train, y_test              = train_test_split(X, y, train_size = 0.8, stratify = y)
normalised_X_train, means_train, ranges_train = createNormalisedFeatures(X_train, "range")
normalised_X_train                            = np.array(normalised_X_train)
normalised_X_test                             = (X_test - means_train)/ranges_train

print("Amount of samples per digit train:")
display(pd.DataFrame(np.unique(y_train, return_counts = True)))
print("Amount of samples per digit test:")
display(pd.DataFrame(np.unique(y_test, return_counts = True)))





## Performing multiclass (one-vs-all) logistic regression on MNIST

Okay, now it is up to you to perform multiclass logistic regression. It's the exact same procedure as before. Here are the instructions again if you need them:

* Make a list of thetas `finalThetasEachDigit`, then use `normalised_X_train` and `y_train` to:
* Loop over the labels (there should be 10 iterations)
* In each loop:
    * Make a new class array, that is 1 for the current digit under consideration, and 0 for all others (use the training data).
    * Use this class array to train logistic regression. Rather than using gradient descent, use `fmin_bfgs`.
    * Append the final thetas to the list.
* When that's done, use `np.c_[yourListName].T` to gather your thetas into a matrix $\Theta$. It should have 65 rows (for the 65 parameters (intercept + 64 pixel intensities)) and 10 columns (one for each digit).
* Make predictions on the test data `normalised_X_test` and `y_test`. Now there's a catch: turns out there's cases where the predicton for, say, the digit being a 3 is _exactly equal_ to the prediction that the digit is an 8. This requires you to use `np.argmax(yourPredictions, axis = 1)`. There's a subtlety here that `argmax` always returns the index of the _first_ maximum element, so you'd always predict 3 in these cases, never 8 which is not _quite_ fair, but those are details we won't bother with here. 
* Finally, there's code below to randomly plot 20 test data digits, which only needs your predicted labels to annotate them with real and predicted labels!

Hints:
* Make sure that your class labels are a 2D array, that is, look like this: $\begin{bmatrix} 0 \\ 1 \\ 1 \\ ... \\ 1 \\ 0 \end{bmatrix}$

In [ ]:
# if you get many warning messages from fmin_bfgs, try this alternative sigmoid function:

# def mySigmoid(data):
#     data = np.array(data)
#     return scipy.special.expit(data)

#Why this? Well, with many parameters and large values, numerical precision for the power function becomes
# an issue. Read here: https://github.com/scipy/scipy/blob/91a279ecb05e7814e2787bfa618d46ad3e0af2be/scipy/special/_logit.h
# how scipy fixes that. 

startThetas = [0]*65 #64 pixels and the intercept parameter


# your answer here



# plotting code, uncomment once you have predictions. Replace predLabelsMNISt with your name for the predictions!

# twentyRandomIndices = np.random.randint(0, len(normalised_X_test), 20)
# fig, axes = plt.subplots(5, 4, figsize = (8,8))
# counter = 0
# for imageRow in range(0,5):
#     for imageCol in range (0,4):
#         axes[imageRow, imageCol].imshow(np.reshape(normalised_X_test[twentyRandomIndices[counter]], newshape = mnistData.images[0].shape),
#                                         cmap='gray', vmin = 0, vmax = 1, interpolation='none')
#         axes[imageRow, imageCol].set_title("Real: " +
#                                            str(y_test[twentyRandomIndices[counter]]) +
#                                            "; pred: " +
#                                            str(predLabelsMNIST[twentyRandomIndices[counter]])
#                                           )
#         counter += 1
# fig.tight_layout()



## Neural networks introduction (~60 minutes)

That's it for logistic regression. Now let's move on to neural networks, which, with the sigmoid activation function we use in them, are layered logistic regressors. In this way, they can extract basic features, then make some intermediate feature representations that help final classification, and then perform that final classification. Before you move on to the actual practical exercises at hand, it probably wouldn't hurt to reinforce what we've covered in the lectures. 3Blue1Brown has four _fantastic_ videos about neural networks. Watch the first 2 now:<br>
* [But what is a neural network?](https://www.youtube.com/watch?v=aircAruvnKk&list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi&index=1)
* [Gradient descent, how neural networks learn](https://www.youtube.com/watch?v=IHZwWFHWa-w&list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi&index=2)

After you're done with that, spend some 15 minutes playing around on the online TensorFlow playground, and try to understand what's going on with the neural network there. Of course, you don't know how it's learning, but you can see how it works visually, which is always a great aid. Be sure to scroll down first and read what blue and orange mean: <br>

* [TensorFlow playground](https://playground.tensorflow.org/#activation=sigmoid&batchSize=10&dataset=gauss&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=0&networkShape=4,3,2&seed=0.86244&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false)




## Implementing feedforward neural networks yourself

The image below was ~shamelessly ripped from~ taken from a practical on forward propagation by Andrew Ng on Coursera. It shows the neural network that we're going to use to perform our predictions with:
![NeuralNetworkImage](NeuralNetwork.PNG)

We'll give our 8\*8 data to the input units, shown in blue. This means that there are 64 input units (excluding the bias unit that always just outputs +1). Just to be clear: what are shown as, and called, input _units_ are just the _feature values_. There's no computation in the blue circles.

Then, there's a Hidden Layer consisting of 25 units (and the bias unit with the +1), and a final layer with 10 output units, that calculate the hypothesis function for each digit (i.e. chance that it is a 0, a 1, a 2, etc.).

You're given two _matrices_ of trained weights, $\Theta_1$ and $\Theta_2$. It'll be up to you to do the computations that finally result in the output of the neural network. Do this computation for the normalised_X_train from above. While we're not going to train the neural network here, remember that we now have our labels as (row) vectors. <br>
So, the digit 0 has as label: $$\begin{bmatrix} 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0  \end{bmatrix}$$
While the digit 5 has as label: $$\begin{bmatrix} 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \end{bmatrix}$$ <br>
Transform the labels in this way using numpy or sklearn (OneHotEncoder) (your call).

So, your task:
* Transform the numeric scalar labels (0-9) into vector labels (see above) using numpy or sklearn. Using OneHotEncoder from sklearn with `sparse = False` and using `.fit_transform(labelsToTransform)` is probably easiest.
* Perform the computations with the two theta matrices such that you go from all the inputs (`normalised_X_train`) to the correct outputs. Don't forget to add bias units (i.e. a 1), and don't forget the activation function (sigmoid)! Make sure to do this with linear algebra: no loops! Finally: the HL has 25 units and the output layer 10 units.
* Once done, make the outcomes classifications (by setting the maximal outcome of the 10 output units as 1, and the rest as 0 in the predicted label)
* Then, check how many train samples are correctly predicted by these trained weight matrices (check whether the output labels are completely equal to the predicted labels).
* Once you're done, check the answers for some code to plot the misclassified samples and their predicted class.

Hints:
* If you get errors about needing to reshape, do exactly as they say.
* The Theta matrices contain the bias and the weights per unit in rows. So: the first row of $\Theta_1$ contains the bias in the first column, and then the 64 weights for the 64 pixels in the following columns.
* You might need to transpose (`.T`) things here to get the desired behaviour. 
* Check the slides for inspiration
* Remember to use `@` for matrix multiplication
* You can use `np.c_[]` in combination with `np.ones()` to append a column of all 1 to the inputs to a layer. Note: to make np.ones  return a column vector, make sure to set its `shape` argument to `(someDimension, 1)`
* To change the outcomes to class vectors, you can loop over the samples, set the maximum to 1 and the rest to 0, or do everything together with `np.where()` and `np.amax(theData, axis = 1)`.
* You might need `np.all(classLabelsPredicted == realClassLabels, axis = 1)`


In [ ]:
thetaOneMatrix = np.loadtxt("thetaOneMatrixFFNN.csv", delimiter = ",")
thetaTwoMatrix = np.loadtxt("thetaTwoMatrixFFNN.csv", delimiter = ",")

#Your code here. Use normalised_X_train!






Repeat of the network image for your convenience:
![NeuralNetworkImage](NeuralNetwork.PNG)

## Nested cross-validation

There's one thing I may have already mentioned but you haven't implemented yet, and that's nested cross-validation. What does it do? Well, remember those pesky hyperparameters like $\lambda$, the regularisation strength, and $\alpha$, the learning rate? I promised I'd let you know how to pick them. The most common way is through this nested cross-validation approach.

**Cross-validation recap** <br>
In cross-validation, you divide your dataset in k folds. You train on k-1 folds k times, and test on the kth fold k times. The average of this gives you a pretty good idea of your generalisation error. The purpose is not to use the k classifiers you trained further and pick the best among them or something, the purpose is to know how well you do on unseen data. <img src="cross-validation.png" alt="Drawing" style="width: 500px;"/>

**Hyperparameter optimisation and nested cross-validation** <br>
Let's say we want to pick our learning rate. The naive approach would be to try 5 learning rates (say 0.001, 0.01, 0.1, 1, 10). You could, in your cross-validation, train 5 classifiers on each fold, one with each learning rate, and look at their average performance on the test folds. Then you could pick the $\alpha$ that's best and be done with it. **That's wrong**. This is because you pick which learning rate is best based on how it performs on the test set: so you're choosing the hyperparameters that do best on your 'unseen' test set, and then reporting that average performance. You will hence be overconfident!

So what to do? Well: nested-cross validation. You make k-1 folds to train on. _Within_ those k-1 folds, you _again_ divide into folds, say l folds. You train 5 classifiers on each of the l-1 folds, one for each value of $\alpha$. Then you test on the l-th fold. The $\alpha$ that perform best on average is the one you pick from among these sub-folds. Then, you move one level up again, and train using that alpha on the whole _upper_ fold of size k-1, and test using it on the k-th fold. Here is an image to further clarify:
<img src="nested-crossval.png" alt="Drawing" style="width: 760px;"/>

In the end, the whole goal is simple: you want to pick your hyperparameters based on _one_ unseen part of the data, and then measure your generalisation performance on some _other_ unseen part of the data that the hyperparameters were not fitted to, so you don't get a positive bias of your generalisation potential. 

Now there's one little detail: this means you end up with k folds that were each, potentially, trained with different hyperparameters. Perhaps on fold 3, an $\alpha$ of 0.01 was best, but on fold 5 one of 0.001 was best. For your _final model_ you could then pick either the hyperparameter that is picked in most of you k folds, or you could take an average.

**Normalisation** <br>
To be entirely correct, you need to normalise the data separately within each inner fold. Otherwise signal from the test data (influence on the mean and variance) can leak into your training data. See 'data transformation with held out data' [here](https://scikit-learn.org/stable/modules/cross_validation.html). You'd really need an sklearn PipeLine for that, which is too advanced for now. So let's forget this for our current purposes.

**Your job** <br>
Whew, The Great Wall of Text, such a tourist attraction! We could implement this completely ourselves, using the k-fold cross-validation function we made earlier in a nested fashion. For now, however, I believe that if you understand the principle, it is fine to implement it with sklearn. Read [this tutorial](https://inria.github.io/scikit-learn-mooc/python_scripts/cross_validation_nested.html) and adapt it to what you need to do below.
Up to you to:
* Use the data below to train regularised _linear regression_ . Now to make matters confusing, sklearn's implementation calls the regularisation parameter **_alpha_**. L2-regularised linear regression is called `Ridge`.
* You'll need to instantiate two k-fold splits `KFold()`. One for the inner, one for the outer. Make the outer 10, and the inner 5.
* Use `GridSearchCV` to train for 5 different values of lambda\alpha (given in a list below). Set `return_train_score = True`. In this way, `YourGridSearchCVObject.cv_results_` will show you the scores obtained for each of the 5 $\lambda$ parameters you use.
* Use `cross_validate` and **not** `cross_val_score`. With cross_validate, you will be able to access all the nested cross-validations performed. Make sure to set `return_estimator = True`.
* Use `for estimator in yourCrossValidateOutput: print(estimator.cv_results_)` to get a look-see inside. You should see that each estimator has 5 train scores and 5 test scores for each split (also 5 splits): 1 for each of the possible hyperparameters. The mean train score tells you how each of the 5 hyperparameters fared over the l folds. You'll see for this simple problem that 0.001 and 0.01 are equally good (and perfect).

Hints
* Use the tutorial I linked in the text above, then exchange cross_val_score for cross_val, and you're basically there.
* This sample ML problem is laughably easy and will give 100% performance, although for larger $\lambda$ values the score will start to suffer (underfitting).
* If this is too unclear, please tell me and I will go over it in class/we'll do it together.

In [ ]:
from sklearn.datasets import make_regression
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, GridSearchCV,  cross_validate

features, values = make_regression(n_samples = 500, n_features = 2, n_informative = 2, bias = 1.33,
                             random_state = 84)

#Show the data
dataFrameReg = pd.concat([pd.DataFrame(features), pd.DataFrame(values[:,np.newaxis])], axis = 1)
dataFrameReg.columns   = ["Feature 1", "Feature 2", "Value"]
print("Example values:")
display(dataFrameReg.head())

#Make a plot of the training data
cmap = sns.cubehelix_palette(as_cmap=True)
fig, ax = plt.subplots(figsize = (7,7))
points = ax.scatter(features[:,0], features[:,1], c = values, cmap = cmap)
ax.set_xlabel("Feature 1")
ax.set_ylabel("Feature 2")
fig.colorbar(points)

param_grid = {"alpha": [0.001, 0.01, 0.1, 1, 10]}


# your answer here




## Result

What you might realise is that we're now training a lot of classifiers. That is, you train k classifiers for k-fold cross-validation, but with internal cross-validation for hyperparameters, you add l internal folds * n_parameter settings classifiers for _each_ outer fold k. So above, we trained a cool 250 classifiers for internal cross_validation (5 internal folds * 5 parameter settings * 10 outer folds k) and 10 for the final outer cross validation (the k folds). You can imagine that with more parameter possibilities you want to try out, things start to get prohibitive.

For that, there's also random search, which, rather than trying out _every combination_ of a set of hyperparameters you want to try out, just tries out _n_ random values in intervals you pick. Turns out that also works pretty well.

As explained before, when training your final classifier on all your training data, and testing it on held-out test data that you kept separate from _the beginning of your project_, you can see which hyperparameters were picked to train with on the k outer folds in the end, and take an average for each hyperparameter, or the value that occurs most.

## What comes now can be freely picked

With such an enticing sub-heading I assume you're basically foaming at the mouth to pick what you want to do. Turns out, 'freely' was slightly overstated and there's basically four options:

1. Finish what might not have gotten finished before, any practical exercises from yesterday or earlier today that you didn't quite get to. <br> <br>
2. Read or watch more about what we discussed these two days to review (or preview the next few days): consult the course reader for recommendations. <br> <br>
3. Do the sneak preview exercise on scikit-learn below. You might have seen me use it to make some datasets, and you'll be using it next week, so you could use this time to get a little hands-on practice in already. <br> <br>
4. Leave this place of learning, so to gain more free time to fill with whatever pleasure and/or debauchery suits your whims!

**Whatever you do, _first_ make sure to scroll down and 1. read the takeaways and 2. fill out the survey!**

## Optional further scikit-learn preview

Hi there, welcome to this optional part that's optional. Did I mention it's optional? Wouldn't want to overwelm anyone with **optional** materials. Without further ado:

Yesterday you traded some of your precious little time on this planet to painstakingly implement gradient descent for linear regression yourself, and you did so this morning for logistic regression too. Turns out that's a sucker's game, because extremely cool and knowledgeable people have already put together a whole library that implements all sorts of ML algorithms, which allows you to train classifiers with very little code. 

Note that sklearn's predictors have methods that you'll work with to fit them to data and to predict on unseen data. As an example, a Random Forest implementation would look something like this: <br> <br>
`
myRandomForest = sklearn.ensemble.RandomForestClassifier(n_estimators = 80, max_depth = 2)
myRandomForest.fit(trainingFeatures, trainingLabels)
#predict class labels
myRandomForest.predict(testFeatures)
#predict class probabilities
myRandomForest.predict_proba(testFeatures)
#see the cost function/score (mean accuracy here)
myRandomForest.score(testFeatures, testLabels)
` <br> <br>
Your first order of business is to implement linear regression.   To do this:

* Import sklearn 
* Load the California housing dataset (look in [sklearn.datasets](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.datasets)). Be sure to do `from sklearn import datasets` as well!
* Print exactly what the targets mean (what are the units of what you're predicting?), what the number of samples is, and use `DataFrame.describe()` to see the mean and distribution of the features (you'll need to convert to a DataFrame for that).
* Split the normalised data into train and test sets ( 80%, 20%, look [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)).
* Normalise both features and the target to have zero mean and unit variance. Use [StandardScaler](https://scikit-learn.org/stable/modules/preprocessing.html#standardization-or-mean-removal-and-variance-scaling). **Note: If you normalise train and test data together, information about the mean and variance of the test data 'leaks' into the transformed data: you've used aspects of the test data in transforming the train data. The correct way is to transform the train data, and then use the mean and variance calculated there to transform the test data. The StandardScaler link shows how to do this**
* Train linear regression without regularisation, and test on the test set. Print the $R^2$ (use `.score`). Also print the learned parameters/weights (use `.coef_`).
* Train linear regression with L2 regularisation (penalising the sum of squares of the parameters). This is called Ridge regression. Set its $\lambda$ (which is confusingly called alpha) to 1.  Test on the test set and print the $R^2$ (use `.score`). Also print the learned parameters/weights (use `.coef_`).
* Train linear regression with L1 regularisation (penalising the sum of the absolutes of the parameters). This is called LASSO regression. Set its $\lambda$ (which is confusingly called alpha) to 0.01. Test on the test set and print the $R^2$ (use `.score`). Also print the learned parameters/weights (use `.coef_`).

Hints:
* It seems like a lot of steps, but it's all basically one-liners in sklearn (okay some are three-liners).
* Search the documentation for the right predictor or functionality. You'll have to learn to navigate it anyway!
* If you don't know what $R^2$ is, see [this](https://www.youtube.com/watch?v=2AQKmw14mHM).

In [ ]:
#your answer here
import sklearn




## Quick sklearn wrap-up
As you can appreciate, training classifiers in this way is pretty easy. Thank open-source developers who implement these routines in an optimised and uniformly usable way. We'll be using sklearn here and there throughout the week, and you'll switch to it for your project next week.

## What I want you to remember here:
* You know how to implement multiclass logistic regression
* You understand how to pick your final predictions: you take the maximum from separate classifiers trained per class
* You understand that neural networks using the sigmoid function are really just layered logistic regressors. By virtue of their layers, they can learn more complex features (as illustrated in the lectures with Boolean functions).
* You understand the use of the sigmoid activation function.
* You know how to implement forward propagation using linear algebra, and what forward propagation means and does.
* You understand nested-cross validation, and see why it is necessary.

## The end

Well done! You've done multiclass logistic regression and made your first steps towards understanding and implementing neural networks. You've earned yourself the definition of the word [niffer](https://www.merriam-webster.com/dictionary/niffer).

## Survey
Who could forget? [Click](https://docs.google.com/forms/d/e/1FAIpQLSfnHr02yXCsyrUTfTeMCEay3wOZLOquY8FuQMIsnkopL_1PFQ/viewform?usp=sf_link)